In [1]:
%pylab inline
import cv2
import numpy as np
import tensorflow as tf

Populating the interactive namespace from numpy and matplotlib


In [2]:
tfrecord = "/media/ryan/UBUNTU 16_0/train_sfm.tfrecord"
all_poses = []

In [3]:
def parser(record):
    keys_to_features = {
        "pose": tf.FixedLenFeature((), tf.string, default_value=""),
        "img_raw": tf.FixedLenFeature((), tf.string, default_value=""),
        "img_raw_prev": tf.FixedLenFeature((), tf.string, default_value=""),
    }
    parsed = tf.parse_single_example(record, keys_to_features)

    # Perform additional preprocessing on the parsed data.
    image = tf.image.decode_jpeg(parsed["img_raw"])
    image = tf.reshape(image, [128, 416, 1])
    image_prev = tf.image.decode_jpeg(parsed["img_raw_prev"])
    image_prev = tf.reshape(image_prev, [128, 416, 1])
    
    label = tf.decode_raw(parsed["pose"], tf.float64)
    label = tf.reshape(label, [12])

    return image, image_prev, label

In [ ]:
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(parser)
dataset = dataset.batch(32)
iterator = dataset.make_initializable_iterator()
all_poses = []
with tf.Session() as sess:
    # Initialize 'iterator' with training data
    training_filenames = ['/media/ryan/UBUNTU 16_0/train_sfm.tfrecord']
    sess.run(iterator.initializer, feed_dict={filenames: training_filenames})
    for i in range(int(18500/32)):
        images, images_prev, labels = iterator.get_next()
        images, images_prev, labels = sess.run([images, images_prev, labels])
        labels = labels.tolist()
        all_poses += labels
    

In [ ]:
print (len(all_poses))
all_poses_arr = np.array(all_poses)
print (all_poses_arr.shape)

In [ ]:
from sklearn.preprocessing import *

In [ ]:
mean = np.mean(all_poses_arr)
std = np.std(all_poses_arr)
print (mean, std)

In [ ]:
all_poses_scaled = all_poses_arr
all_poses_scaled -= mean
all_poses_scaled /= std

In [ ]:
print (all_poses_scaled)

In [ ]:
scaler = StandardScaler().fit(all_poses_arr)
all_poses_scikit = scaler.transform(all_poses_arr)

In [ ]:
print (all_poses_scikit)

In [ ]:
print (scaler.mean_)
print (scaler.scale_)

In [ ]:
import seaborn as sns
sns.set(color_codes=True)
sns.distplot(all_poses_scikit[:,0])

In [ ]:
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
print (np.array(data).shape)

In [ ]:
import pickle as p

with open('pose_scaler.p', 'wb') as fp:
    p.dump(scaler, fp)

In [ ]:
filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(parser)
dataset = dataset.batch(1)
dataset = dataset.shuffle(buffer_size=500)
iterator = dataset.make_initializable_iterator()
all_images = []
with tf.Session() as sess:
    # Initialize 'iterator' with training data
    training_filenames = ['/media/ryan/UBUNTU 16_0/train_sfm.tfrecord']
    sess.run(iterator.initializer, feed_dict={filenames: training_filenames})
    for i in range(int(5000)):
        if i %500 == 0:
            print (i)
        images, images_prev, labels = iterator.get_next()
        images, images_prev, labels = sess.run([images, images_prev, labels])
        all_images.append(images)

0
500
1000
1500
2000
2500
3000


In [ ]:
all_images_arr = np.array(all_images).astype('float')
print (all_images_arr.shape)

In [ ]:
all_images_arr = all_images_arr.squeeze()
print (all_images_arr.shape)

In [ ]:
mean = np.mean(all_images_arr)

In [ ]:
std = np.std(all_images_arr)

In [ ]:
print (mean, std)

In [ ]:
all_images_arr -= mean
all_images_arr /= std
print (all_images_arr[0])